In [1]:
import uproot
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import progressbar
import multiprocessing as mp
#from tqdm import *

In [2]:
calcValues = True

In [3]:
mainPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/TEST_FOLDER_MASS/"
testFoldersTmp = []
for filename in sorted(os.listdir(mainPath)):
    testFoldersTmp.append(filename)
testFolders = (np.sort(np.array(testFoldersTmp).astype(int))).astype(str)

## Berechnung der Integralmatrix 

In [4]:
def prepareArray(origArray):
    preparedArray = np.zeros(len(origArray), dtype=complex)
    for i in range(0, len(origArray)):
        preparedArray[i] = origArray[i][0]
    return preparedArray

In [5]:
def calcErrors(folderI):
    #if bar.currval <= folderI:
    #    bar.update(folderI)
    ampPath = mainPath + testFolders[folderI] + '/amps/'
    onlyfiles = []
    for filename in sorted(os.listdir(ampPath)):
        onlyfiles.append(filename)
    nmbWaves = len(onlyfiles)
    ampFile1 = uproot.open(ampPath + onlyfiles[0])
    nmbEvents = len(ampFile1[ampFile1.keys()[0]]['amplitude'].get('_incohSubAmps._real').array())

    amp = np.zeros((nmbWaves, nmbWaves), dtype = complex)
    sd = np.zeros((nmbWaves, nmbWaves), dtype = complex)
    ampMatrix = np.zeros((nmbWaves,nmbEvents), dtype = complex)



    for waveIndexI in range(0, 1): #nmbWaves):
        if ampMatrix[waveIndexI].all() == 0:
            ampFile1 = uproot.open(ampPath + onlyfiles[waveIndexI])
            key1 = ampFile1.keys()[0]

            incohSubAmpsReal1 = ampFile1[key1]['amplitude'].get('_incohSubAmps._real').array()
            incohSubAmpsImag1 = ampFile1[key1]['amplitude'].get('_incohSubAmps._imag').array()
            ampMatrix[waveIndexI] = prepareArray(np.array(incohSubAmpsReal1 + 1j * incohSubAmpsImag1))
            #print ampMatrix[waveIndexI]
            #raise ValueError('Exit.')




        for waveIndexJ in range(0, 2): # waveIndexI + 1): #nmbWaves):
            if ampMatrix[waveIndexJ].all() == 0:
                ampFile2 = uproot.open(ampPath + onlyfiles[waveIndexJ])
                key2 = ampFile2.keys()[0]

                incohSubAmpsReal2 = ampFile2[key2]['amplitude'].get('_incohSubAmps._real').array()
                incohSubAmpsImag2 = ampFile2[key2]['amplitude'].get('_incohSubAmps._imag').array()
                ampMatrix[waveIndexJ] = prepareArray(np.array(incohSubAmpsReal2 + 1j * incohSubAmpsImag2))

            integral = np.sum(ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate())
            amp[waveIndexI][waveIndexJ] = amp[waveIndexJ][waveIndexI]

            sdReal = sum(pow((ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate()).real, 2)).real
            sdImag = sum(pow((ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate()).imag, 2)).real
            #print "Var1:", sdReal

            sdReal = math.sqrt((sdReal - pow(amp[waveIndexI][waveIndexJ].real,2)/float(nmbEvents))/float(nmbEvents))
            sdImag = math.sqrt((sdImag - pow(amp[waveIndexI][waveIndexJ].imag,2)/float(nmbEvents))/float(nmbEvents))
            sdComplex = complex(sdReal, sdImag)        
            errorComplex = complex(sdReal/math.sqrt(nmbEvents), sdImag/math.sqrt(nmbEvents))

            sd[waveIndexI][waveIndexJ] = sdComplex
            sd[waveIndexJ][waveIndexI] = sdComplex

            amp[waveIndexI][waveIndexJ] = integral
            amp[waveIndexJ][waveIndexI] = integral      
            #print "%i, %i, %i: %s, %s, %s, %s" %(folderI, waveIndexI, waveIndexJ, amp[waveIndexI][waveIndexJ], amp[waveIndexI][waveIndexJ]/nmbEvents, sd[waveIndexI][waveIndexJ], errorComplex)
            #print np.cov(amp/nmbEvents)
            #raise ValueError('Exit.')

            errorMatrixAbsolut[waveIndexI][waveIndexJ][folderI] = errorComplex
            errorMatrixAbsolut[waveIndexJ][waveIndexI][folderI] = errorComplex

            if waveIndexI == waveIndexJ:
                errorMatrixRelativ[waveIndexI][waveIndexJ][folderI] = complex(errorComplex.real/math.fabs(integral.real), 0)
            else:
                errorMatrixRelativ[waveIndexI][waveIndexJ][folderI] = complex(errorComplex.real/math.fabs(integral.real), errorComplex.imag/math.fabs(integral.imag))
                errorMatrixRelativ[waveIndexJ][waveIndexI][folderI] = complex(errorComplex.real/math.fabs(integral.real), errorComplex.imag/math.fabs(integral.imag))
    return (folderI, errorMatrixAbsolut, errorMatrixRelativ, testFolders[folderI])


In [ ]:
def test(folderI):
    print folderI

In [7]:
nmbTestFolders = len(testFolders)
ampPathTmp = mainPath + testFolders[0] + '/amps/'
onlyfiles = []
for filename in sorted(os.listdir(ampPathTmp)):
    onlyfiles.append(filename)
nmbWaves = len(onlyfiles)


bar = progressbar.ProgressBar(maxval=nmbTestFolders, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
pool = mp.Pool(2)#mp.cpu_count())
results = pool.map(calcErrors, [folderI for folderI in range(0,nmbTestFolders)])
'''
with tqdm(total=nmbTestFolders) as pbar:
    #for i, res in tqdm(enumerate(pool.imap_unordered(calcErrors, [folderI for folderI in range(0,nmbTestFolders)]))):
    for i, res in tqdm(enumerate(pool.map(calcErrors, [folderI for folderI in range(0,nmbTestFolders)]))):
        results.append(res) 
        pbar.update()
'''
pool.close()   
pool.join()
bar.update(bar.maxval)



#nmbWaves = len([name for name in os.listdir(mainPath + testFolders[0] + '/amps/') if os.path.isfile(os.path.join(mainPath + testFolders[0] + '/amps/', name))])
axisArr = np.zeros(nmbTestFolders, dtype=int)
errorMatrixAbsolut = np.zeros((nmbWaves, nmbWaves), dtype = np.ndarray)
errorMatrixRelativ = np.zeros((nmbWaves, nmbWaves), dtype = np.ndarray)
for i in range(0, nmbWaves):
    for j in range(0, nmbWaves):
        errorMatrixAbsolut[i][j] = np.zeros(nmbTestFolders, dtype=complex)
        errorMatrixRelativ[i][j] = np.zeros(nmbTestFolders, dtype=complex)
        
for resultI in range(0,len(results)):
    folderI = results[resultI][0]
    axisArr[folderI] = results[resultI][3]
    for waveIndexI in range(0, nmbWaves):
        for waveIndexJ in range(0, waveIndexI + 1):
            errorMatrixAbsolut[waveIndexI][waveIndexJ][folderI] = results[resultI][1][waveIndexI][waveIndexJ][folderI]
            errorMatrixAbsolut[waveIndexI][waveIndexJ][folderI] = results[resultI][1][waveIndexJ][waveIndexI][folderI]
            errorMatrixRelativ[waveIndexI][waveIndexJ][folderI] = results[resultI][2][waveIndexI][waveIndexJ][folderI]
            errorMatrixRelativ[waveIndexI][waveIndexJ][folderI] = results[resultI][2][waveIndexJ][waveIndexI][folderI]


'''
data = np.zeros(len(errorMatrixAbsolut), dtype = [('mass', int), ('fileName', 'U100'), ('absolutError', np.ndarray), ('relativError', np.ndarray)])
for resultI in range(0,len(results)):
    folderI = results[resultI][0]
    axisArr[folderI] = results[resultI][3]
    for waveIndexI in range(0, nmbWaves):
        for waveIndexJ in range(0, waveIndexI + 1):
            #data[folderI] = (results[resultsI][3], , results[resultsI][1], results[resultsI][2])
'''       
#onlyfiles = results[0][4]

print "Finish successfully"

NameError: global name 'errorMatrixAbsolut' is not defined

In [ ]:
if calcValues == False:
    for i in range(0,5): #len(testFolders)):
        for j in range(0, i+1): #len(testFolders)):
            print i, j
            fig = plt.figure(figsize=(10,3))
            plt.grid(True)
            plt.xticks(np.arange(0, max(axisArr), 1000))
            plt.plot(axisArr, errorMatrixAbsolut[i][j].real, 'o')
            #plt.plot(axisArr, errorMatrixRelativ[i][j].real*100, 'o')
            #print errorMatrix[i][j]
            print onlyfiles[i]
            print onlyfiles[j]
            plt.xlabel('Mass [MeV/c^2]')
            plt.ylabel('Error [%]')
            plt.show()
            
            
            if i != j:
                fig = plt.figure(figsize=(10,3))
                plt.grid(True)
                plt.xticks(np.arange(0, max(axisArr), 1000))
                plt.plot(axisArr, errorMatrixAbsolut[i][j].imag, 'ro')
                #plt.plot(axisArr, errorMatrixRelativ[i][j].imag*100, 'ro')
                #print errorMatrix[i][j]
                print onlyfiles[i]
                print onlyfiles[j]
                plt.xlabel('Mass [MeV/c^2]')
                plt.ylabel('Error [%]')
                plt.show()


## Test Bereich

In [ ]:
#if calcValues == True:
    #np.save('Fehler_bei_unterschiedlicher_Masse_Auswertung_Absolut.npy', errorMatrixAbsolut)
    #np.save('Fehler_bei_unterschiedlicher_Masse_Auswertung_Relativ.npy', errorMatrixRelativ)

In [ ]:
# %reset

In [ ]:
# print len(errorMatrixAbsolut[0][1].real), len(errorMatrixAbsolut[0][1].imag)